This is a tutorial to teach you how to use our program to optimize the portfolio 
and the price prediction.

This tutorial consist of 7 steps:

    step 0 :to collect price data using yfiance
    step 1 :to predict the stock price using GP
    step 2 :to predict the stock price using fusion GP
    
    

step 0 :to collect price data using yfiance

Now we are going to collect the daily price data.
In order to reduce computational cost, we just select three stocks as the research target.

These Stocks are : AAPL,NVDA,TSLA

we want to collect the stock prices between 01.01.2010 and 01.01.2020.

At first , let us collec the daily prices and store them in folder "demo_data/short".

It means that the daily prices data is the short-term data.


In [1]:
pip install yfinance --user


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install ruamel-yaml


Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install PyPortfolioOpt --user

Note: you may need to restart the kernel to use updated packages.


In [25]:
import sys
sys.path.append("..")


In [27]:
import data_set_price.data_collect as data_collect

import yfinance

stock_id_list=["AAPL","NVDA","TSLA"]

In [29]:
data_collect.download_data(stock_id_list,interval="1d",start="2010-01-01",
                  end="2020-01-01",
                  target_diretory="demo_data/short",
                  back_adjust=True,auto_adjust=True)

And then , we need to generate the long-term data.
The long-term data is the monthly stock prices.
we store them in folder "demo_data/long".

Now our short-term data set and long-term data set are ready

step 1: to predict price using GP

 Here we try to predict the monthly price of "AAPL" on 01.02.2015 using GPR.
  
  The training data set for the fitting is the monthly stock prices between 01.01.2010 and 01.01.2015.
  

In [31]:
import util
import gp_wrapper_prices as gp_prices

D:\anaconda\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.decorator.check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(
D:\anaconda\lib\site-packages\gpflow\experimental\utils.py:42: UserWarning: You're calling gpflow.experimental.check_shapes.inheritance.inherit_check_shapes which is considered *experimental*. Expect: breaking changes, poor documentation, and bugs.
  warn(


In [ ]:
start_time = "2010-01-01 00:00:00"
end_time = "2015-01-01 00:00:00"
start = util.convert_time_into_datetime(time=start_time)
end = util.convert_time_into_datetime(time=end_time)
gp_price_AAPL=gp_prices.gp_wrapper_price(csv_file="demo_data/long/AAPL.csv")
X_pred_times,Y_pred_mean,Y_pred_cov,Y_pred_actual,loss_score=gp_price_AAPL.predict(start_time=start,end_time=end,pred_length=1)

X_pred_times is the timestamp

In [9]:
X_pred_times 

array(['2015-02-01T00:00:00.000000000'], dtype='datetime64[ns]')

Y_pred_mean is the predicted result.

In [10]:
Y_pred_mean

array([23.98002998])

Y_pred_actual is the actual value.

In [11]:
Y_pred_actual

array([28.78534126])

loss_score is the error.

Now loss_score = abs(Y_pred_actual -Y_pred_mean)

In [12]:
loss_score

4.80531128672002

Now let us shorten the size of the tarining set.

set start_time is on 01.01.2014

In [13]:
start_time = "2014-01-01 00:00:00"
end_time = "2015-01-01 00:00:00"
start = util.convert_time_into_datetime(time=start_time)
end = util.convert_time_into_datetime(time=end_time)
gp_price_AAPL=gp_prices.gp_wrapper_price(csv_file="demo_data/long/AAPL.csv")
X_pred_times,Y_pred_mean,Y_pred_cov,Y_pred_actual,loss_score=gp_price_AAPL.predict(start_time=start,end_time=end,pred_length=1)

In [14]:
X_pred_times

array(['2015-02-01T00:00:00.000000000'], dtype='datetime64[ns]')

In [15]:
Y_pred_mean

array([25.1109809])

In [16]:
Y_pred_actual

array([28.78534126])

In [17]:
loss_score

3.6743603615021243

As we can see: 
The loss_score becomes samller while the time interval of the tarining data becomes smaller
But a very smaller time interval lead to that the amount of sample points is very small.
That is reason why we need to fuse the long-term data and short-term data.
Because the short-term data still can offer a lot of sample points within a small time interval.

 step 2: to predict price using fusion GP
 
 Here we try to predict the monthly price of "AAPL" on 01.02.2015 using fusion GP.

The long-term data set for the fitting is the monthly stock prices between 01.01.2010 and 01.01.2015.

The short-term data set for the fitting is the daily stock prices between 01.01.2010 and 01.01.2015.

In [18]:
import fusion_wrapper_price

In [19]:
fusion_price_AAPL = fusion_wrapper_price.fusion_wrapper_price(csv_path_long_term="demo_data/long/AAPL.csv",
                                                              csv_path_short_term="demo_data/short/AAPL.csv")

In [20]:
start_time = "2010-01-01 00:00:00"
end_time = "2015-01-01 00:00:00"
start3 = util.convert_time_into_datetime(time=start_time)
end3 = util.convert_time_into_datetime(time=end_time)

In [21]:
 X_time,Y_mean_fusion,Y_pred_mean_long,Y_pred_actual_long,error_fusion,error_long=fusion_price_AAPL.fusion_next_price(start=start3,end=end3,forward_window=15)


D:\anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


X_time is the timstampe of the predicted result.

Y_mean_fusion is the predicted fusion result.

Y_pred_mean_long is the predicted result without fusion.(just to predict the long-term price like step 1)

Y_pred_actual_long is the actual long-term price. 

error_fusion is : error_fusion = (Y_mean_fusion -Y_pred_actual_long)

error_long is: error_long= (Y_pred_mean_long -Y_pred_actual_long)

Here we care error_fusion and error_long.

If abs(error_fusion) < abs(error_long) , we can say that the fusion is better than the directly prediction.




In [22]:
X_time

Timestamp('2015-02-01 00:00:00')

In [23]:
Y_mean_fusion

array([24.48446362])

In [24]:
Y_pred_mean_long

array([23.98002998])

In [25]:
Y_pred_actual_long

array([28.78534126])

In [26]:
error_fusion

4.3008776469518715

In [27]:
error_long

4.80531128672002

it seems that error_long > error_fusion

At least now ,we can say the fusion is better.

Step 3 : to generate the return and the portfolios

in step 0 , we have generated the short-term data , the daily stock price , and the long-term data , the monthly stock price.

In portfolios optimization , we can generate a portfolios based on the historical return.

Now , we need generate the historical returns from the stock prices.


In [28]:
from portfolios_generation import Portfolios

In order to reduce computational cost, we don't generate the whole historical return.

we just select a smaller time interval [01.06.2013  ,  01.06.2018].



In [29]:
start_time = "2013-06-01 00:00:00"
end_time = "2018-06-01 00:00:00"
start=util.convert_time_into_datetime(start_time)
end=util.convert_time_into_datetime(end_time)

we need to generate two different historical returns ,one for long-term and one for short-term

In [30]:
long_term_path_dataset="demo_data/long"
short_term_path_dataset="demo_data/short"
long_term_portfolio=Portfolios(long_term_path_dataset)
short_term_portfolio=Portfolios(short_term_path_dataset)

In [31]:
# load the price data between [start_time,end_time]
long_term_portfolio.load_historical_price(start=start,end=end)
short_term_portfolio.load_historical_price(start=start,end=end)

,AAPL,NVDA,TSLA
Date,,,
2013-06-03,13.956647,3.373095,6.172667
2013-06-04,13.912984,3.377763,6.322667
2013-06-05,13.782929,3.305399,6.358000
2013-06-06,13.577014,3.326407,6.490000
2013-06-07,13.680744,3.373095,6.802667
...,...,...,...
2018-05-25,44.994549,61.745781,18.590000
2018-05-29,44.832302,61.574863,18.917334
2018-05-30,44.736870,62.664730,19.448000


In [32]:
# generate the historical return
long_term_historical_return=long_term_portfolio.generate_historical_returns()
short_term_historical_return=short_term_portfolio.generate_historical_returns()

In [33]:
# generate the historical portfolios
portfolio_long= long_term_portfolio.generate_historical_portfolios(start=start,end=end,forward_length=12)
portfolio_short= short_term_portfolio.generate_historical_portfolios(start=start,end=end,forward_length=12)


assdsd
2014-06-01 00:00:00
mu AAPL    0.072717
NVDA    0.028696
TSLA   -0.000577
Name: 2014-05-01 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  1.429181  0.083587 -0.442027
NVDA  0.083587  1.116530  1.112869
TSLA -0.442027  1.112869  7.722953
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2013-07-01  0.141225  0.028490  0.250745
2013-08-01  0.076658  0.021468  0.258564
2013-09-01 -0.015028  0.060234  0.144201
2013-10-01  0.096381 -0.023778 -0.172881
2013-11-01  0.063842  0.026991 -0.204202
2013-12-01  0.014792  0.032485  0.181883
2014-01-01 -0.107697 -0.019975  0.205943
2014-02-01  0.051219  0.170700  0.349485
2014-03-01  0.026057 -0.021171 -0.148523
2014-04-01  0.099397  0.031268 -0.002687
2014-05-01  0.072717  0.028696 -0.000577
Expected annual return: 3.4%
Annual semi-deviation: 31.6%
Sortino Ratio: 0.05
2014-07-01 00:00:00
mu AAPL    0.033402
NVDA   -0.019717
TSLA    0.155412
Name: 2014-06-01 00:00:00, dtype: fl

Expected annual return: 0.4%
Annual semi-deviation: 47.1%
Sortino Ratio: -0.03
2015-10-01 00:00:00
mu AAPL   -0.017388
NVDA    0.101144
TSLA   -0.002650
Name: 2015-09-01 00:00:00, dtype: float64
S          AAPL     NVDA     TSLA
AAPL  1.30595  0.00000  0.00000
NVDA  0.00000  1.30595  0.00000
TSLA  0.00000  0.00000  1.30595
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2014-10-01  0.071960  0.059079 -0.004038
2014-11-01  0.101204  0.073183  0.011667
2014-12-01 -0.067867 -0.039826 -0.090422
2015-01-01  0.061424 -0.042394 -0.084574
2015-02-01  0.096449  0.148958 -0.001277
2015-03-01 -0.027549 -0.047571 -0.071653
2015-04-01  0.005787  0.060679  0.197489
2015-05-01  0.040990 -0.003154  0.109489
2015-06-01 -0.033206 -0.087072  0.069617
2015-07-01 -0.032927 -0.007956 -0.007866
2015-08-01 -0.070404  0.126817 -0.064212
2015-09-01 -0.017388  0.101144 -0.002650
Expected annual return: 2.9%
Annual semi-deviation: 40.4%
Sortino Ratio: 0.02
2015-11-0

2016-07-01 00:00:00
mu AAPL   -0.036831
NVDA    0.008820
TSLA   -0.049053
Name: 2016-06-01 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  2.309032  0.000000  0.000000
NVDA  0.000000  2.309032  0.000000
TSLA  0.000000  0.000000  2.309032
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2015-07-01 -0.032927 -0.007956 -0.007866
2015-08-01 -0.070404  0.126817 -0.064212
2015-09-01 -0.017388  0.101144 -0.002650
2015-10-01  0.083409  0.150913 -0.166948
2015-11-01 -0.010042  0.118083  0.112743
2015-12-01 -0.106420  0.043038  0.042343
2016-01-01 -0.075242 -0.111347 -0.203367
2016-02-01 -0.006678  0.070672  0.003818
2016-03-01  0.133327  0.140300  0.197155
2016-04-01 -0.139921 -0.002807  0.047830
2016-05-01  0.065287  0.314945 -0.072811
2016-06-01 -0.036831  0.008820 -0.049053
Expected annual return: -2.5%
Annual semi-deviation: 62.7%
Sortino Ratio: -0.07
2016-08-01 00:00:00
mu AAPL    0.090063
NVDA    0.214635
TSLA    0.106039


Expected annual return: 8.1%
Annual semi-deviation: 18.3%
Sortino Ratio: 0.33
2017-05-01 00:00:00
mu AAPL   -0.000069
NVDA   -0.042505
TSLA    0.128530
Name: 2017-04-01 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  1.390684 -0.010231  0.106041
NVDA -0.010231  2.762050 -0.284459
TSLA  0.106041 -0.284459  2.112379
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2016-05-01  0.065287  0.314945 -0.072811
2016-06-01 -0.036831  0.008820 -0.049053
2016-07-01  0.090063  0.214635  0.106039
2016-08-01  0.018136  0.074256 -0.097023
2016-09-01  0.071276  0.119111 -0.037640
2016-10-01  0.004334  0.038529 -0.030878
2016-11-01 -0.026599  0.295672 -0.042128
2016-12-01  0.053336  0.159434  0.128247
2017-01-01  0.047747  0.022859  0.178951
2017-02-01  0.128884 -0.070526 -0.007701
2017-03-01  0.053236  0.074768  0.113245
2017-04-01 -0.000069 -0.042505  0.128530
Expected annual return: 3.3%
Annual semi-deviation: 12.5%
Sortino Ratio: 0.1

C:\Users\Archer\AppData\Roaming\Python\Python38\site-packages\pypfopt\efficient_frontier\efficient_semivariance.py:275: RuntimeWarning: divide by zero encountered in double_scalars
  sortino_ratio = (mu - risk_free_rate) / semi_deviation
C:\Users\Archer\AppData\Roaming\Python\Python38\site-packages\pypfopt\efficient_frontier\efficient_semivariance.py:275: RuntimeWarning: divide by zero encountered in double_scalars
  sortino_ratio = (mu - risk_free_rate) / semi_deviation


...

TSLA  0.013015  0.022633  0.115252
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2018-02-27 -0.003241 -0.002109 -0.017990
2018-02-28 -0.001514 -0.016500 -0.022593
2018-03-01 -0.017516 -0.040454 -0.035358
2018-03-02  0.006914  0.018647  0.012661
2018-03-05  0.003462 -0.003763 -0.005282
2018-03-06 -0.000848  0.027626 -0.015449
2018-03-07 -0.009283 -0.001321  0.012492
2018-03-08  0.010913 -0.002729 -0.009630
2018-03-09  0.017181  0.017207 -0.005865
2018-03-12  0.009668  0.018057  0.056057
2018-03-13 -0.009630 -0.008208 -0.010622
2018-03-14 -0.008501  0.004158 -0.044494
Expected annual return: -1.6%
Annual semi-deviation: 18.1%
Sortino Ratio: -0.20
2018-03-16 00:00:00
mu AAPL    0.001177
NVDA    0.002412
TSLA   -0.003153
Name: 2018-03-15 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  0.059823  0.011442  0.012499
NVDA  0.011442  0.082375  0.021954
TSLA  0.012499  0.021954  0.113138
stock_returns                 AAPL     

Expected annual return: -0.5%
Annual semi-deviation: 41.4%
Sortino Ratio: -0.06
2018-04-11 00:00:00
mu AAPL    0.018818
NVDA    0.058029
TSLA    0.051923
Name: 2018-04-10 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  0.156902  0.146519  0.144846
NVDA  0.146519  0.408952  0.293045
TSLA  0.144846  0.293045  0.671767
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2018-03-23 -0.023157 -0.036717 -0.024458
2018-03-26  0.047472  0.049406  0.008755
2018-03-27 -0.025641 -0.077553 -0.082188
2018-03-28 -0.011049 -0.018490 -0.076653
2018-03-29  0.007809  0.046262  0.032392
2018-04-02 -0.006556 -0.045511 -0.051291
2018-04-03  0.010259  0.019453  0.059609
2018-04-04  0.019122  0.003949  0.072553
2018-04-05  0.006934 -0.021482  0.065449
2018-04-06 -0.025579 -0.032207 -0.021000
2018-04-09  0.009918  0.005414 -0.032208
2018-04-10  0.018818  0.058029  0.051923
Expected annual return: 4.3%
Annual semi-deviation: 40.6%
Sortino Ratio: 0

Expected annual return: 1.8%
Annual semi-deviation: 16.2%
Sortino Ratio: -0.01
2018-05-03 00:00:00
mu AAPL    0.044175
NVDA   -0.003654
TSLA    0.004101
Name: 2018-05-02 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  0.101938  0.000000  0.000000
NVDA  0.000000  0.101938  0.000000
TSLA  0.000000  0.000000  0.101938
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2018-04-17  0.013764  0.026135 -0.012088
2018-04-18 -0.002244 -0.004925  0.019674
2018-04-19 -0.028340 -0.031011  0.022942
2018-04-20 -0.040972 -0.001441 -0.032791
2018-04-23 -0.002896 -0.021118 -0.023670
2018-04-24 -0.013919 -0.011971  0.000318
2018-04-25  0.004357 -0.020524 -0.009772
2018-04-26  0.003483  0.039509  0.017065
2018-04-27 -0.011570  0.004929  0.030125
2018-04-30  0.018112 -0.006318 -0.000612
2018-05-01  0.023236  0.009960  0.020483
2018-05-02  0.044175 -0.003654  0.004101
Expected annual return: 1.5%
Annual semi-deviation: 15.8%
Sortino Ratio: -0

Expected annual return: -1.0%
Annual semi-deviation: 8.1%
Sortino Ratio: -0.36
2018-05-16 00:00:00
mu AAPL   -0.009088
NVDA   -0.038377
TSLA   -0.026681
Name: 2018-05-15 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  0.096046  0.008218  0.020188
NVDA  0.008218  0.113310  0.016198
TSLA  0.020188  0.016198  0.128124
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2018-04-30  0.018112 -0.006318 -0.000612
2018-05-01  0.023236  0.009960  0.020483
2018-05-02  0.044175 -0.003654  0.004101
2018-05-03  0.001812  0.029517 -0.055454
2018-05-04  0.039234  0.026053  0.033890
2018-05-07  0.007235  0.040241  0.029515
2018-05-08  0.004807  0.006917 -0.002642
2018-05-09  0.007041  0.021486  0.016161
2018-05-10  0.014304  0.017007 -0.005964
2018-05-11 -0.003803 -0.021528 -0.012983
2018-05-14 -0.002333  0.003261 -0.030193
2018-05-15 -0.009088 -0.038377 -0.026681
Expected annual return: -2.5%
Annual semi-deviation: 14.0%
Sortino Ratio: -

Expected annual return: 0.4%
Annual semi-deviation: 16.1%
Sortino Ratio: -0.10
2018-05-30 00:00:00
mu AAPL   -0.003606
NVDA   -0.002768
TSLA    0.017608
Name: 2018-05-29 00:00:00, dtype: float64
S           AAPL      NVDA      TSLA
AAPL  0.020687  0.007685  0.012756
NVDA  0.007685  0.051746  0.020870
TSLA  0.012756  0.020870  0.084654
stock_returns                 AAPL      NVDA      TSLA
Date                                    
2018-05-11 -0.003803 -0.021528 -0.012983
2018-05-14 -0.002333  0.003261 -0.030193
2018-05-15 -0.009088 -0.038377 -0.026681
2018-05-16  0.009333  0.001833  0.008093
2018-05-17 -0.006324  0.006910 -0.006772
2018-05-18 -0.003637 -0.007146 -0.027132
2018-05-21  0.007085 -0.006912  0.027708
2018-05-22 -0.002505 -0.006305 -0.033323
2018-05-23  0.006412  0.020573  0.014763
2018-05-24 -0.001115  0.000606 -0.004372
2018-05-25  0.002285  0.006419  0.003599
2018-05-29 -0.003606 -0.002768  0.017608
Expected annual return: 0.4%
Annual semi-deviation: 16.1%
Sortino Ratio: -0

forward_length is the data set size used for the portfolios generation.

For example , forward_length =3.
we generate a monthly historical portfolio in 03.2016.

It means that the data set (the monthly stock prices) used in portfolio generation just contains 3 samples , the monthly price in 03.2016,the monthly price in 02.2016 and the monthly price in 01.2016.


Finally, we need to compute the profits of these historical portfolios.

In [34]:
long_term_profit= long_term_portfolio.compute_portfolios_profit()
short_term_profit= short_term_portfolio.compute_portfolios_profit()


And then we need to store the historical returns and the historical portfolios as the csv files in folder "demo_data" .

In [36]:

long_term_historical_return.to_csv("demo_data/historical_return_long.csv", date_format='%Y-%m-%d %X')

short_term_historical_return.to_csv("demo_data/historical_return_short.csv", date_format='%Y-%m-%d %X')

portfolio_long.to_csv("demo_data/historical_portfolios_long.csv", date_format='%Y-%m-%d %X')

portfolio_short.to_csv("demo_data/historical_portfolios_short.csv", date_format='%Y-%m-%d %X')


step4: to predict the long-term portfolio in 09.2017 using GP.

Here , we are going to predict the long-term portfolio using GPR.
we fit the GPR with the long-term historical portfolio between time interval [2014-08  ,2017-08  ]

In [37]:
from gp_wrapper_portfolio import gp_wrapper_portfolio

In [45]:
start_time = "2014-08-01 00:00:00"
end_time = "2017-08-01 00:00:00"

start=util.convert_time_into_datetime(time=start_time)
end = util.convert_time_into_datetime(time=end_time)
long_term_portfolios_path="demo_data/historical_portfolios_long.csv"
long_term_returns_path="demo_data/historical_return_long.csv"

long_portfolio_GP = gp_wrapper_portfolio(data_file_portfolio=long_term_portfolios_path,
                                         data_file_return=long_term_returns_path)

actual_portfolio,actual_profit,pred_portfolio,pred_profit,loss_score=long_portfolio_GP.predict(start_time=start,end_time=end, pred_length=1,with_loss=False)



score is [[0. 0. 0. 0.]]


D:\anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


actual_portfolio : The acutal portfolio in 09.2017

actual_profit : The profit of the acutal portfolio in 09.2017

pred_portfolio : The predicted portfolio in 09.2017

pred_profit : The profit of the predicted portfolio in 09.2017



In [48]:
actual_portfolio


,AAPL,NVDA,TSLA
2017-09-01,0.338536,0.323531,0.337933


In [49]:
actual_profit

,AAPL,NVDA,TSLA,Profit
2017-09-01,-0.019146,0.018115,-0.014053,-0.015083


In [50]:
pred_portfolio

,AAPL,NVDA,TSLA
2017-09-01,0.328693,0.343613,0.327694


In [51]:
pred_profit

,AAPL,NVDA,TSLA,Profit
2017-09-01,-0.018589,0.01924,-0.013627,-0.012976



As we can see , the pred_profit is better the actual_profit.

The actual_profit is -0.015083.
The pred_profit is -0.012976.
Although both of them are the negative return.
But the loss of the pred_portfolio is smaller than the loss of the actual_portfolio. 


step 5: to predict the long-term portfolio in 09.2017 using fusion GP.

Here , we are going to predict the long-term portfolio using fusion GP.


The long-term data set for the fitting is the monthly stock prices between 01.08.2014 and 01.06.2017.

The short-term data set for the fitting is the daily stock prices between 01.08.2014 and 01.06.2017.





In [52]:
from fusion_wrapper_portfolio import fusion_wrapper_portfolio

In [56]:
start_time = "2014-08-01 00:00:00"
end_time = "2017-08-01 00:00:00"

start=util.convert_time_into_datetime(time=start_time)
end = util.convert_time_into_datetime(time=end_time)

long_term_portfolios_path="demo_data/historical_portfolios_long.csv"
long_term_returns_path="demo_data/historical_return_long.csv"
short_term_portfolios_path="demo_data/historical_portfolios_short.csv"
short_term_returns_path="demo_data/historical_return_short.csv"

fusion=fusion_wrapper_portfolio(return_long_term=long_term_returns_path,return_short_term=short_term_returns_path,
            portfolio_long_term=long_term_portfolios_path,portfolio_short_term=short_term_portfolios_path)

In [58]:
actual_portfolio_l, fusion_portfolio, actual_profit_l, fusion_profit, fusion_error, pred_long_error=fusion.fusion_next_portfolio(start=start, end=end, forward_window=14)


D:\anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


score is 2017-09-01    0.002107
Name: Profit, dtype: float64


D:\anaconda\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


score is 2017-08-22   -0.000016
Name: Profit, dtype: float64


In [ ]:
actual_portfolio_l: The acutal long-term portfolio in 09.2017
    
fusion_portfolio: The fused long-term portfolio in 09.2017
    
actual_profit_l: The profit of the acutal long-term portfolio in 09.2017
    
fusion_profit: The profit of the fused long-term portfolio  in 09.2017
    


In [59]:
actual_portfolio_l

,AAPL,NVDA,TSLA
2017-09-01,0.338536,0.323531,0.337933


In [60]:
fusion_portfolio

,AAPL,NVDA,TSLA
2017-09-01,0.333136,0.333738,0.333126


In [61]:
actual_profit_l

,AAPL,NVDA,TSLA,Profit
2017-09-01,-0.019146,0.018115,-0.014053,-0.015083


In [62]:
fusion_profit

,AAPL,NVDA,TSLA,Profit
2017-09-01,-0.01884,0.018687,-0.013853,-0.014006


As we can see ,the fusion_profit is better than the actual_profit_l.


,AAPL,NVDA,TSLA,Profit
2017-09-01,0.000557,0.001124,0.000426,0.002107
